In [67]:
texts = []
import nltk
for i in nltk.corpus.gutenberg.fileids():
    txt = nltk.corpus.gutenberg.words(i)
    texts.append(" ".join(list(txt)))

In [68]:
len(texts)

18

In [69]:
#load photo files
with open("photo.txt") as ph:
    photo = ph.read()
texts.append(photo)
len(texts)

19

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(texts)

In [72]:
tfidf_vectorizer.vocabulary_
nonsparse = tfidf.toarray()
list(tfidf_vectorizer.vocabulary_.items())[0]

('unshut', 39467)

In [81]:
#sorted(list(nonsparse[18,:]), reverse=True)
photodict = {}
for i,j in tfidf_vectorizer.vocabulary_.items():
    
    photodict[i] = nonsparse[18,:][j]
[i.capitalize() if i[0] == 'r' or i[0] == 'p' else i for i in list(photodict.keys()) ]

['Paean',
 'slippered',
 'bird',
 'unshut',
 'zoned',
 'evanescent',
 'bluster',
 'fusing',
 'multum',
 'negroes',
 'whisperer',
 'Pleasantness',
 'Pathological',
 'Propheticke',
 'Ranting',
 'antipathy',
 'astounded',
 'knee',
 'extrication',
 'joshaviah',
 'catastrophe',
 'dancing',
 'Primary',
 'stirre',
 'shaved',
 'visionary',
 'washest',
 'encroachment',
 'gay',
 'Pattering',
 'conforming',
 'seyton',
 'outbreak',
 'strangled',
 'dathan',
 'stun',
 'tilgathpilneser',
 'korahites',
 'Papa',
 'heare',
 'Pronouns',
 'masculine',
 'Prophecies',
 'orders',
 'sute',
 'teareth',
 'backwardness',
 'lovingest',
 'scul',
 'yankees',
 'clan',
 'sickens',
 'beaker',
 'chili',
 'flutter',
 'jimnah',
 'anted',
 'spearings',
 'Rustick',
 'mourne',
 'wrinkles',
 'counted',
 'borrower',
 'thawed',
 'shortened',
 'corruptly',
 'twining',
 'battery',
 'illimitably',
 'cummerbund',
 'armes',
 'quelled',
 'vanities',
 'bestead',
 'untainted',
 'excesses',
 'abounds',
 'Pelt',
 'Poysond',
 'Pouredst',

In [74]:
items = photodict.items()
items = list(items)

import pandas as pd
df = pd.DataFrame(items, columns=["word", "score"])
df = df.sort('score', ascending=False)
df = df[:500]
df
#df.to_csv("photo_terms.csv")

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,score
3717,daguerre,0.274648
26141,photography,0.267605
3013,mr,0.264171
32702,photographic,0.218310
4090,process,0.216622
34056,camera,0.169014
16740,light,0.150195
29932,plate,0.149914
3479,________________,0.140845
38026,daguerreotype,0.133803


In [41]:
import requests 
import enchant

words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words')
stoplist1 = words.text.split("\r\n")

from nltk.corpus import stopwords
stoplist2 = set(stopwords.words('english'))

stoplist1.extend(stoplist2)

fullstops = list(set(stoplist1))
def remove_stops(stoplist, wordlist):
    result = []
    for i in wordlist:
        if i not in stoplist:
                result.append(i)
    return result

def spellcheck(wordlist):
    result = []
    d = enchant.Dict("en_US")
    for i in wordlist:
        if d.check(i) or d.check(i.capitalize()):
            result.append(i)
         
    return result
def clean_text(list_of_texts):
    fully_cleaned =[]
    #normalize ocr errors
    for i in list_of_texts:
        #lowercase all
        ocr_lower = i.lower()
        #tokenize, remove punctuation and numbers, remove tabs, newlines, etc.
        ocr_cleaner = ocr_lower.replace("\n", " ").replace("\t", " ")
        ocr_tokens = ocr_cleaner.split(" ")
        
        no_numbers_or_punct = []
        for token in ocr_tokens:
            if token.isalpha():
                no_numbers_or_punct.append(token)
            else:
                
                new_token = ""
                for letter in token:
                    if letter.isalpha():
                        new_token += letter
                if new_token != "":
                    no_numbers_or_punct.append(new_token)  
        almost_ready_before_spellcheck = remove_stops(fullstops, no_numbers_or_punct)
        almost_ready = spellcheck(almost_ready_before_spellcheck)
        from nltk.stem import WordNetLemmatizer
        #lemmatization
        wordnet_lemmatizer = WordNetLemmatizer()
        lemmas = []
        for token in almost_ready:
            lemma = wordnet_lemmatizer.lemmatize(token)
            lemmas.append(lemma)
        
        ready = [i for i in lemmas if len(i) > 2]
        fully_cleaned.append(ready)
    return fully_cleaned 

In [42]:
photo_cleaned = clean_text([photo])

In [43]:
#term counts
from collections import Counter
photo_counts = Counter(photo_cleaned[0])


In [44]:
photo_counts

Counter({'simpler': 1,
         'deal': 1,
         'property': 6,
         'scarcely': 5,
         'evanescent': 2,
         'easy': 2,
         'holocaust': 1,
         'newspaper': 1,
         'accepted': 2,
         'agreement': 1,
         'recess': 1,
         'effecting': 1,
         'surface': 9,
         'entirely': 3,
         'shall': 7,
         'superior': 2,
         'conceive': 1,
         'defied': 1,
         'portico': 1,
         'amateur': 4,
         'wedgwood': 4,
         'call': 1,
         'tends': 1,
         'conceives': 2,
         'relation': 3,
         'mist': 1,
         'bristled': 1,
         'asylum': 3,
         'engaged': 7,
         'ascertain': 1,
         'unable': 2,
         'amiable': 2,
         'rage': 1,
         'requires': 6,
         'reaching': 1,
         'recently': 3,
         'principle': 6,
         'contact': 4,
         'polished': 2,
         'compact': 1,
         'invalidate': 1,
         'unalterable': 1,
         'access': 1

In [51]:
from random import shuffle
from application.selective_features import make_feature_list, dictionaries_of_features, make_genres_big_and_lavin
from application.pickles import pickledData
import pandas as pd
from scipy.stats.stats import spearmanr
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import recall_score

pData = pickledData()

# import feature dictionaries
feature_dicts_full = pData.feature_dicts
feature_dicts_full.append(dict(photo_counts))
photo_terms = make_feature_list("lavin_lexicon/photo_terms.csv", "score", 500)

#perform feature selection on dictionaries of term frequencies, using top 100 photo_terms
feature_dicts = dictionaries_of_features(feature_dicts_full, photo_terms)

#define a dictionary with ids, genres, dates, authors, etc. Pass to each function
metadata = {"ids": pData._ids, "dates": pData.dates, "genres":pData.genres, "authors":pData.authors,
                "titles":pData.titles,"big_genres":pData.big_genres, "lavin_genres":pData.lavin_genres}

#convert all metadata to dataframe
data = [metadata[i] for i in metadata.keys()]
df = pd.DataFrame(data)
df = df.transpose()
df.index.name = 'position'
df.columns=metadata.keys()


Loaded pickle data successfully.


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=False)
D = [dict(i) for i in feature_dicts]
X = v.fit_transform(D)


In [53]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X)
all_texts = tfidf.toarray()

In [62]:
cosines = []
from sklearn.metrics.pairwise import cosine_similarity
for i in all_texts:
    s = cosine_similarity(i, all_texts[-1])
    cosines.append(float(s))

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [63]:
cosines

[0.1403004156464109,
 0.168515511656509,
 0.16388242214097912,
 0.10974464464533785,
 0.19210649010350125,
 0.159107348360559,
 0.07897286574623291,
 0.1628902526660581,
 0.11204004945733755,
 0.1680642749010907,
 0.17034877690309128,
 0.15383046246589888,
 0.17789458728623445,
 0.15865935797764985,
 0.15109157356848527,
 0.14282161017026695,
 0.1714604928631,
 0.18983896573983755,
 0.13754223962924536,
 0.19757118968375956,
 0.17487016671969013,
 0.11657777602053174,
 0.16480273149491023,
 0.12480344896769059,
 0.16687376862066705,
 0.15195532994613367,
 0.16707009322754918,
 0.1545391425355865,
 0.15921694838315018,
 0.1586903028552206,
 0.1633943418624492,
 0.15920382308843067,
 0.12704570421285738,
 0.18656720785639977,
 0.21381942555876157,
 0.19773510617113735,
 0.13206627323276027,
 0.16430174673066955,
 0.1680291311286544,
 0.08052504956122981,
 0.10952988094096294,
 0.1253431611661571,
 0.13756653016367865,
 0.1917589976340841,
 0.16930551873863658,
 0.149069952931104,
 0.1437

In [64]:
df['cosine_similarities'] = cosines[0:-1]

In [66]:
df.to_csv("photo_similarity_scores.csv")